In [ ]:
#!ollama pull llama3.1


pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕                ▏    0 B/4.7 GB                  

In [ ]:
# !pip install git+https://github.com/openai/whisper.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/whisper.git to c:\users\wolle\appdata\local\temp\pip-req-build-jyww3kkt
  Resolved https://github.com/openai/whisper.git to commit fc5ded7d9045c693692f13853857c3f8baea3a7b
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\wolle\AppData\Local\Temp\pip-req-build-jyww3kkt'


In [3]:
import whisper
from pydub import AudioSegment
import ollama
from deep_translator import GoogleTranslator
import os

# Ensure Ollama is running locally
ollama_client = ollama.Client()

def process_audio(input_audio_path, llama_model="llama3.1"):
    """
    Processes an audio file and returns the transcription, detected language,
    translated text (if applicable), and Llama's response.
    
    Args:
        input_audio_path (str): Path to the input audio file.
        llama_model (str): Name of the Llama model for generating responses.
        
    Returns:
        dict: Contains the transcription, detected language, translated text,
              and Llama's response.
    """
    try:
        # Step 1: Convert input audio to WAV format
        converted_audio_path = "temp_audio.wav"
        audio = AudioSegment.from_file(input_audio_path)
        audio.export(converted_audio_path, format="wav")

        # Step 2: Transcribe the WAV audio file using Whisper
        model = whisper.load_model("medium")
        transcription_result = model.transcribe(converted_audio_path)
        transcription = transcription_result["text"]
        detected_language = transcription_result["language"]

        # Step 3: Translate to English if the detected language is not English
        if detected_language != "en":
            translated_text = GoogleTranslator(source=detected_language, target="en").translate(transcription)
        else:
            translated_text = transcription

        # Step 4: Pass the translated text to Llama (via Ollama)
        response = ollama_client.chat(
            model=llama_model,
            messages=[{"role": "user", "content": translated_text}]
        )
        llama_response = response["message"]["content"] if "message" in response and "content" in response["message"] else "No response from Llama."

        # Clean up temporary audio file
        if os.path.exists(converted_audio_path):
            os.remove(converted_audio_path)

        return {
            "transcription": transcription,
            "detected_language": detected_language,
            "translated_text": translated_text,
            "llama_response": llama_response,
        }

    except Exception as e:
        return {"error": str(e)}

# Example usage
if __name__ == "__main__":
    input_audio_path = r"D:\quat\chat\Nao_medical_backend\test_aud.mp3"
    result = process_audio(input_audio_path)
    if "error" in result:
        print(f"Error processing audio: {result['error']}")
    else:
        print(f"Transcription: {result['transcription']}")
        print(f"Detected Language: {result['detected_language']}")
        print(f"Translated Text: {result['translated_text']}")
        print(f"Llama's Response:\n{result['llama_response']}")


Transcription:  पेशंट का ब्लॉट प्रेशर हाई है और उन्हें द्वाई रोज सूबा लेने की हिडायात दी गेई है।
Detected Language: hi
Translated Text: The patient's blood pressure is high and he has been advised to take medicine daily.
Llama's Response:
It sounds like the patient has high blood pressure, also known as hypertension. This can be a serious condition if not managed properly.

The doctor's advice for the patient to take medicine daily is likely to help lower their blood pressure and prevent any potential complications. Is there anything else you'd like to know about this situation or would you like some general information on managing high blood pressure?
